In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.metrics import classification_report, confusion_matrix
import os, datetime, random, warnings

# 🚫 Suppress Warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# ====== Reproducibility ======
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

# ====== Load and Preprocess Data ======
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train[..., np.newaxis] / 255.0, x_test[..., np.newaxis] / 255.0

AUTOTUNE = tf.data.AUTOTUNE
batch_size = 64

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def preprocess(x, y):
    y = tf.one_hot(y, depth=10)
    return tf.cast(x, tf.float32), y

train_ds = (train_ds
            .shuffle(10000)
            .map(preprocess, num_parallel_calls=AUTOTUNE)
            .batch(batch_size)
            .prefetch(AUTOTUNE))

test_ds = (test_ds
           .map(preprocess, num_parallel_calls=AUTOTUNE)
           .batch(batch_size)
           .prefetch(AUTOTUNE))

# ====== Build Model with GlobalAveragePooling and BatchNorm ======
def build_model():
    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

model = build_model()

# ====== Compile Model with Label Smoothing ======
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy'])

# ====== Setup Callbacks ======
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
cb_list = [
    callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2),
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_accuracy", save_best_only=True),
    callbacks.TensorBoard(log_dir=log_dir)
]

# ====== Train Model ======
model.fit(train_ds,
          epochs=20,
          validation_data=test_ds,
          callbacks=cb_list)

# ====== Evaluate Model ======
loss, acc = model.evaluate(test_ds)
print(f"✅ Final Test Accuracy: {acc:.4f}")

# ====== Detailed Classification Metrics ======
y_pred = model.predict(test_ds)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

print("
Classification Report:")
print(classification_report(y_true, y_pred_classes))

print("
Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

# ====== Save Full Model ======
model.save("mnist_final_model.keras")
print("✅ Full model saved to 'mnist_final_model.keras'")

# ====== Convert to TFLite for Deployment ======
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("mnist_model.tflite", "wb") as f:
    f.write(tflite_model)
print("✅ Model converted and saved as 'mnist_model.tflite'")